In [11]:
import psycopg2
import os
import pandas as pd
import json

In [18]:
postgres_usr = os.environ.get('POSTGRES_USR')
postgres_pwd = os.environ.get('POSTGRES_PWD')
postgres_host = os.environ.get('POSTGRES_HOST')

conn_uri = f'postgresql://{postgres_usr}:{postgres_pwd}@{postgres_host}:5432/postgres'

In [8]:
# Connect to your PostgreSQL database using cursor
# Execute DDL
def connect_database_ddl(exec_query:str, db, usr, pwd, host):
    conn = psycopg2.connect(f"dbname={db} user={usr} password={pwd} host={host}")
    cursor = conn.cursor()
    # Execute the TRUNCATE statement
    cursor.execute(exec_query)
    # Commit the changes
    conn.commit()
    # Close the cursor and connection
    cursor.close()
    conn.close()

In [9]:
exec_query = "TRUNCATE TABLE retail.orders"
connect_database_ddl(exec_query, "postgres", postgres_usr, postgres_pwd, postgres_host)

In [32]:
"""
Pandas read_sql can not execute DDL
"""

query = 'SELECT * FROM retail.orders'
pd.read_sql(query, conn_uri)

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE
...,...,...,...,...
68878,68879,2014-07-09 00:00:00.0,778,COMPLETE
68879,68880,2014-07-13 00:00:00.0,1117,COMPLETE
68880,68881,2014-07-19 00:00:00.0,2518,PENDING_PAYMENT
68881,68882,2014-07-22 00:00:00.0,10000,ON_HOLD


In [12]:
def get_column_names(schemas: dict, ds_name: str, sorting_key='column_position'):
    """
    Get column names from schemas.json
    """
    column_details = schemas[ds_name]
    columns = sorted(column_details, key=lambda col: col[sorting_key], reverse=False)
    return [col['column_name'] for col in columns]

In [15]:
schemas = json.load(open('../data/retail_db/schemas.json'))
columns = get_column_names(schemas, 'orders')
df = pd.read_csv('../data/retail_db/orders/part-00000', names=columns)

In [30]:
df.to_sql(
    name='orders',
    con=conn_uri,
    schema='retail',
    if_exists='replace',
    index=False
)

883

In [31]:
#help(df.to_sql)

In [23]:
df

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE
...,...,...,...,...
68878,68879,2014-07-09 00:00:00.0,778,COMPLETE
68879,68880,2014-07-13 00:00:00.0,1117,COMPLETE
68880,68881,2014-07-19 00:00:00.0,2518,PENDING_PAYMENT
68881,68882,2014-07-22 00:00:00.0,10000,ON_HOLD
